# GPT API Introduction

In this notebook, we'll explore the basics of using the OpenAI GPT API, starting from the very beginning with setting up the environment and connecting to the API.

## 1. Setting Up Your OpenAI Account

Before using the OpenAI API, you need to create an account and obtain an API key. Follow these steps:
1. Visit the [OpenAI website](https://openai.com/) and sign up for an account.
2. Verify your email address to activate your account.
3. Navigate to the API section of your account dashboard.
4. Click on "Create API Key" to generate a new token.
5. Copy the generated token and store it securely, as it will be used to authenticate your API requests.

> ### Understanding Usage Charges
> 
> The pricing for the GPT-3.5-turbo model is **$0.002 per 1,000 tokens**. 
> - **What is a Token?** A token can be as short as one character or as long as one word. On average, 1,000 tokens correspond to about 750 words.
> - **Cost Examples**:
>   - **100 tokens**: ~$0.0002 (about 75 words)
>   - **500 tokens**: ~$0.001 (about 375 words)
>   - **1,000 tokens**: ~$0.002 (about 750 words)
>   - **5,000 tokens**: ~$0.01 (about 3,750 words)
> 
> This makes it affordable to experiment with the API without incurring significant expenses. Always check the [OpenAI pricing page](https://openai.com/pricing) for the most up-to-date information on usage charges.

## 2. Setting up the Environment

First, we need to ensure that we have the necessary packages installed. The main package we'll be using is `openai`. Let's create a function to check if it's installed and install it if necessary.

In [ ]:
import importlib.util
import subprocess
import sys
import os
from getpass import getpass

def check_openai_installed():
    if importlib.util.find_spec("openai") is None:
        print("OpenAI package not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "openai"])
        print("OpenAI package installed successfully.")
    else:
        print("OpenAI package is already installed.")

check_openai_installed()

## 3. Importing the OpenAI Library and Setting up the API Key

Now that we've ensured the `openai` package is installed, let's import it and set up the API key.

To use the OpenAI API, we need an API key. It's important to keep this key secure. If you cloned the repository locally, you can simply add your API key to a `.env` file in the root directory of the project. This way, you won't need to re-enter your API key each time you run the notebook.

> **Note on Security**: When entering your API key in Jupyter notebooks or Google Colab, be cautious. Avoid sharing your notebooks publicly or exposing your API key in any way. Always ensure that your API key is kept private to prevent unauthorized access to your OpenAI account.

In [ ]:
from openai import OpenAI

def validate_api_key(api_key):
    try:
        client = OpenAI(api_key=api_key)
        # Make a simple API call to test the key
        client.models.list()
        return True
    except Exception as e:
        print(f"Error validating API key: {str(e)}")
        return False

def get_api_key():
    while True:
        api_key = os.getenv("OPENAI_API_KEY")
        if not api_key:
            print("OPENAI_API_KEY not found in environment variables.")
            api_key = getpass("Please enter your OpenAI API key: ")
        
        if validate_api_key(api_key):
            os.environ["OPENAI_API_KEY"] = api_key
            return api_key
        else:
            print("Invalid API key. Please try again.")
            os.environ.pop("OPENAI_API_KEY", None)  # Remove the invalid key from env

## 4. Setting up the OpenAI Client

In this step, we will set up the OpenAI client using the validated API key. This client will be used to make requests to the OpenAI API.

In [ ]:
_client = None

def setup_openai_client():
    global _client
    api_key = get_api_key()
    _client = OpenAI(api_key=api_key)
    print(f"API Key set: {'Yes' if _client.api_key else 'No'}")
    print(f"API Key (first 5 chars): {_client.api_key[:5] if _client.api_key else 'None'}")
    return _client

client = setup_openai_client()

## 5. Testing the API Connection

Here, we will test the API connection by making a simple request to ensure that the client is set up correctly and the API key is valid.

In [ ]:
def test_api(client):
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "Hello, World!"}
            ],
            max_tokens=10  # Increase max_tokens for a longer response
        )
        print("API test successful. Response:", response.choices[0].message.content.strip())
    except Exception as e:
        print("API test failed. Error:", str(e))

# Test the API connection
test_api(client)

## 6. Generating Text with GPT

In this step, we will define a function to generate text using the GPT model based on a user-provided prompt.

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    global _client
    if _client is None:
        _client = setup_openai_client()
    messages = [{"role": "user", "content": prompt}]
    response = _client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content

# Test the text generation
prompt = "Write a short poem about artificial intelligence:"
generated_poem = get_completion(prompt)
print("\nGenerated Poem:")
print(generated_poem)

## 7. Helper Function and Minification

In this step, we will create a helper function that encapsulates all the setup code into a minified format for easier reuse in future sessions.

In [ ]:
def create_minified_helper():
    import inspect
    functions = [check_openai_installed, validate_api_key, get_api_key, setup_openai_client, get_completion]
    minified_code = "import importlib,subprocess,sys,os,getpass\n_client=None\n"
    for func in functions:
        code = inspect.getsource(func)
        # Basic minification (remove comments and extra whitespace)
        minified_code += "\n".join(line.strip() for line in code.split("\n") if line.strip() and not line.strip().startswith("#"))
    return f"exec('''{minified_code}''')\nclient=setup_openai_client()"

minified_helper = create_minified_helper()
print("Minified Helper Function:")
print(minified_helper)

This minified helper function combines all the necessary setup code into a single, compact string that can be executed with `exec()`. 

> ### Note on Security
> 
> **Entering Your API Key**: When entering your API key in Jupyter notebooks or Google Colab, be cautious. Avoid sharing your notebooks publicly or exposing your API key in any way. If you cloned the repository locally, you can add your API key to a `.env` file in the root directory to avoid re-entering it each time.
> 
> **Using `exec()`**: Using `exec()` can be a security risk if the code being executed is not trusted. In this case, we're generating the code ourselves, so it's safe. However, always be cautious when using `exec()` with any code that could be influenced by external input.
> 
> **Cross-Platform Support**: By minifying the code and using `exec()`, we avoid issues with complex file paths and imports that can vary across different platforms, making our setup more portable. In private development environments like GitHub Codespaces, the risks of using `exec()` are minimized, as the code runs in a controlled environment.
> 
> **Inspecting Code**: There is no real risk of malicious code, but you can always inspect this code (or have GPT inspect it beforehand).

> **Disclaimer**: The author accepts no responsibility for any misuses of the authored code or any consequences that may arise from its use. Always ensure you understand the code you are executing and use it responsibly.

## Exercises

Now that we have our API connection set up and working, try experimenting with the following tasks to get a better understanding of how to manipulate the parameters of the GPT API:

### 1. Change the Role and Content
Modify the role of the assistant in the prompt. For example, change the role from "user" to "system" or "assistant" and see how the response changes.
- **Hint**: You can adjust the `messages` parameter in the `get_completion` function to include different roles.
- **Example**: 
  ```python
  messages = [
      {"role": "system", "content": "You are a knowledgeable assistant."},
      {"role": "user", "content": "What is the capital of France?"}
  ]
  ```

In [ ]:
# Change the Role and Content
messages = [
    {"role": "system", "content": "You are a knowledgeable assistant."},
    {"role": "user", "content": "What is the capital of France?"}
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    max_tokens=50
)
print(response.choices[0].message.content.strip())

### 2. Adjust the Temperature
Experiment with the `temperature` parameter to see how it affects the creativity of the responses. A lower temperature (e.g., 0.2) will make the output more focused and deterministic, while a higher temperature (e.g., 0.8) will produce more varied and creative responses.
- **Hint**: Modify the `temperature` argument in the `get_completion` function call.

In [ ]:
# Adjust the Temperature
prompt = "Write a creative story about a dragon."
generated_text = get_completion(prompt, temperature=0.5)  # Change the temperature value
print(generated_text)

### 3. Change the Max Tokens
Adjust the `max_tokens` parameter to control the length of the generated response. Try setting it to different values (e.g., 50, 100, 200) and observe how the length and detail of the output change.
- **Hint**: You can modify the `max_tokens` argument in the `get_completion` function.

In [ ]:
# Change the Max Tokens
prompt = "Explain the theory of relativity in simple terms."
generated_text = get_completion(prompt, max_tokens=100)  # Change the max_tokens value
print(generated_text)

Feel free to explore and combine these parameters to see how they interact with each other and affect the output!